In [1]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
# pretrained_model = "distilbert-base-uncased"
# pretrained_model = "xlm-roberta-large"
# pretrained_model = "microsoft/deberta-v3-small"
# pretrained_model = "microsoft/deberta-v3-base"
pretrained_model = "microsoft/deberta-v3-large"
# pretrained_model = "kamalkraj/deberta-v2-xlarge"
# model_checkpoint = "/root/ner-model/test-ner/checkpoint-2500/"

source_dataset_name = "wnut_17"
checkpoint_name = 'checkpoint-1278'
model_checkpoint = f"../data/models/{pretrained_model}/{source_dataset_name}/{checkpoint_name}"

use_pretrained_model = False ## vs. use checkpoint model
if use_pretrained_model:
    model_checkpoint = pretrained_model

batch_size = 24
num_train_epochs = 24

# dataset_name = "wnut_17"
# dataset_name = "conll2003"

dataset_name = "SpeedOfMagic/ontonotes_english"


In [2]:
import neptune.new as neptune
from transformers.integrations import NeptuneCallback

run_name = f'{pretrained_model}' if use_pretrained_model else f'{pretrained_model}/{source_dataset_name}'
tags = [pretrained_model, dataset_name, "NER"]
if not use_pretrained_model:
    tags.append(f'{source_dataset_name}->{dataset_name}')

# Create a Neptune run object
run = neptune.init_run(
    project="thingumajig/ner-training",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJhN2QyZGRkNS0yNDhhLTQyYWItOTg5ZC1kZGQ3Yjc3ZDMxZTcifQ==",
    
    name=f"train-{run_name}:{dataset_name}", description=f"{run_name} fine-tuned on {dataset_name}",
    
    tags=tags,  # 

    source_files=["*.ipynb"],  # 

)

neptune_callback = NeptuneCallback(run=run)

neptune_run_id = run["sys/id"].fetch()
print(f'Run id: {neptune_run_id}')

2022-10-09 19:19:35.808312: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-09 19:19:35.912308: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-09 19:19:36.270528: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-10-09 19:19:36.270572: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

https://app.neptune.ai/thingumajig/ner-training/e/NERTRAIN-5
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.
Run id: NERTRAIN-5


# Loading dataset

In [3]:
from datasets import load_dataset, load_metric

In [4]:
datasets = load_dataset(dataset_name)

Found cached dataset wnut_17 (/root/.cache/huggingface/datasets/wnut_17/wnut_17/1.0.0/077c7f08b8dbc800692e8c9186cdf3606d5849ab0e7be662e6135bb10eba54f9)


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 3394
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1009
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1287
    })
})

In [6]:
datasets["train"][0]

{'id': '0',
 'tokens': ['@paulwalk',
  'It',
  "'s",
  'the',
  'view',
  'from',
  'where',
  'I',
  "'m",
  'living',
  'for',
  'two',
  'weeks',
  '.',
  'Empire',
  'State',
  'Building',
  '=',
  'ESB',
  '.',
  'Pretty',
  'bad',
  'storm',
  'here',
  'last',
  'evening',
  '.'],
 'ner_tags': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  8,
  8,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]}

In [7]:
datasets["train"].features[f"ner_tags"]

Sequence(feature=ClassLabel(num_classes=13, names=['O', 'B-corporation', 'I-corporation', 'B-creative-work', 'I-creative-work', 'B-group', 'I-group', 'B-location', 'I-location', 'B-person', 'I-person', 'B-product', 'I-product'], id=None), length=-1, id=None)

In [8]:
label_list = datasets["train"].features[f"{task}_tags"].feature.names
label_list

['O',
 'B-corporation',
 'I-corporation',
 'B-creative-work',
 'I-creative-work',
 'B-group',
 'I-group',
 'B-location',
 'I-location',
 'B-person',
 'I-person',
 'B-product',
 'I-product']

In [9]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [10]:
show_random_elements(datasets["train"])

,id,tokens,ner_tags
0,3262,"[4th, and, goal, :, Sports, viewership, scores, with, strong, online, moves, :, Television, is, just, one, way, to, consume, sports, ..., http://t.co/HWvMlZHc4j]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
1,2946,"[@MINHOICHOI, @BabySoul__IRP, jan, pacaran, di, mt, q]","[O, O, O, O, O, O, O]"
2,2545,"[@thegreatcoso, @Boavenossa, @EllaTambien06, alejo, vive, en, mar, del, plata, ., Que, puede, saber, de, cosas, del, campo, ?]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
3,1251,"[RT, @FTFDband, NEW, SONG, COMING, THIS, SUNDAY, !, :D]","[O, O, O, O, O, O, O, O, O]"
4,1389,"[@BundleHunt, Is, it, being, released, today, or, soon, ?]","[O, O, O, O, O, O, O, O, O]"
5,2493,"[/I'm, gonna, reply, to, forgotten, mentions, tomorrow, I, promise, !, ~, --, Or, that, night, ,, depends, if, busy, or, not, .]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
6,566,"[$$, #forex, #fx, :, Forex, Articles, for, the, Weekend, September, 18lo, http://www.forexcrunch.com/forex-articles-for-the-weekend-september-18lo/]","[O, O, O, O, B-corporation, O, O, O, O, O, O, O]"
7,412,"[Looks, like, we, will, be, living, in, the, studio, for, the, next, couple, of, months]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
8,3104,"[SUB-CULTURE, is, house, &amp;, bass, inna, Chapel, Hill, town, this, Sat, 1/10, w/, @oneduran, &amp;, @djfmdotcom, at, @CH_underground, https://t.co/eQjU65mW0F]","[O, O, O, O, O, O, B-location, I-location, O, O, O, O, O, O, O, O, O, O, O]"
9,3059,"[@WMonikaFonseca, Happy, Sunday, !]","[O, O, O, O]"


# Preprocessing dataset

In [11]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/root/.cache/pypoetry/virtualenvs/sequence-labeling-1mH2o5_I-py3.8/lib/python3.8/site-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [13]:
tokenizer("Hello, this is one sentence!")

{'input_ids': [1, 5365, 261, 291, 269, 311, 4378, 300, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [14]:
example = datasets["train"][4]
print(example["tokens"])

['4Dbling', "'s", 'place', 'til', 'monday', ',', 'party', 'party', 'party', '.', '&lt;', '3']


In [15]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', '▁4', 'D', 'bling', "▁'", 's', '▁place', '▁til', '▁monday', '▁,', '▁party', '▁party', '▁party', '▁.', '▁&', 'lt', ';', '▁3', '[SEP]']


In [16]:
len(example[f"{task}_tags"]), len(tokenized_input["input_ids"])

(12, 19)

In [17]:
print(tokenized_input.word_ids())

[None, 0, 0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 10, 11, None]


In [18]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"{task}_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

19 19


In [19]:
label_all_tokens = True

In [20]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [21]:
tokenize_and_align_labels(datasets['train'][:5])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': [[1, 1944, 44086, 17328, 325, 382, 268, 262, 866, 292, 399, 273, 382, 358, 821, 270, 375, 1033, 323, 6251, 867, 4098, 1842, 74224, 323, 9452, 966, 4084, 422, 437, 1882, 323, 2], [1, 1007, 2036, 1864, 26742, 877, 22613, 14706, 9149, 5983, 270, 14538, 3005, 264, 4024, 323, 456, 1871, 294, 320, 320, 94698, 260, 549, 320, 2823, 452, 60317, 4921, 2], [1, 916, 43475, 39750, 2482, 960, 9591, 12852, 8794, 265, 1499, 1524, 323, 260, 260, 1871, 294, 320, 320, 5435, 260, 701, 320, 45036, 38842, 3206, 693, 953, 57483, 2], [1, 561, 269, 312, 437, 406, 288, 262, 832, 323, 2], [1, 453, 691, 27020, 382, 268, 470, 17846, 44840, 366, 915, 915, 915, 323, 429, 19011, 346, 404, 2]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0

In [22]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

# Fine-tuning the model

In [23]:
from transformers import AutoConfig, AutoModelForTokenClassification, TrainingArguments, Trainer
# config = AutoConfig.from_pretrained(model_checkpoint)
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list), ignore_mismatched_sizes=True) #, from_tf=True

Some weights of the model checkpoint at microsoft/deberta-v3-large were not used when initializing DebertaV2ForTokenClassification: ['mask_predictions.dense.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.bias', 'deberta.embeddings.position_embeddings.weight', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.LayerNorm.weight', 'mask_predictions.classifier.bias', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.weight']
- This IS expected if you are initializing DebertaV2ForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a B

In [24]:
args = TrainingArguments(
    # f"test-{task}",
    f"../data/models/{pretrained_model}/{dataset_name}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,

    weight_decay=0.01,
    num_train_epochs=num_train_epochs,
    
    save_strategy='epoch',
    save_total_limit = 3,
    
    metric_for_best_model = 'f1',
    load_best_model_at_end=True,
    
    report_to="none",
)

In [25]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [26]:
metric = load_metric("seqeval")
# from evaluate import evaluator

# task_evaluator = evaluator("token-classification")

/tmp/ipykernel_2585073/1092872775.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [27]:
labels = [label_list[i] for i in example[f"{task}_tags"]]
metric.compute(predictions=[labels], references=[labels])

{'person': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [28]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

## Trainer

In [29]:
import torch
torch.cuda.empty_cache()

In [30]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    
    callbacks=[neptune_callback],
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DebertaV2ForTokenClassification.forward` and have been ignored: tokens, ner_tags, id. If tokens, ner_tags, id are not expected by `DebertaV2ForTokenClassification.forward`,  you can safely ignore this message.
/root/.cache/pypoetry/virtualenvs/sequence-labeling-1mH2o5_I-py3.8/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3394
  Num Epochs = 6
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1278
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, usin

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.232225,0.689700,0.465669,0.555964,0.945497


The following columns in the evaluation set don't have a corresponding argument in `DebertaV2ForTokenClassification.forward` and have been ignored: tokens, ner_tags, id. If tokens, ner_tags, id are not expected by `DebertaV2ForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1009
  Batch size = 16
Saving model checkpoint to ../data/models/microsoft/deberta-v3-large/wnut_17/checkpoint-213
Configuration saved in ../data/models/microsoft/deberta-v3-large/wnut_17/checkpoint-213/config.json
Model weights saved in ../data/models/microsoft/deberta-v3-large/wnut_17/checkpoint-213/pytorch_model.bin
tokenizer config file saved in ../data/models/microsoft/deberta-v3-large/wnut_17/checkpoint-213/tokenizer_config.json
Special tokens file saved in ../data/models/microsoft/deberta-v3-large/wnut_17/checkpoint-213/special_tokens_map.json
/root/.cache/pypoetry/virtualenvs/sequence-labeling-1mH2o5_I-py3.8/lib/python3.8/site-packages/transf

In [ ]:
trainer.save_model(f"../data/models/{pretrained_model}/{dataset_name}/final",)

In [ ]:
restored_run = neptune.init_run(
    with_id=neptune_run_id,
    
    project="thingumajig/ner-training",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJhN2QyZGRkNS0yNDhhLTQyYWItOTg5ZC1kZGQ3Yjc3ZDMxZTcifQ==",
)
trainer.pop_callback(neptune_callback)
trainer.add_callback(NeptuneCallback(run=restored_run))

trainer.evaluate()

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

In [ ]:
# trainer.log(results)
restored_run['evaluation_extra'] = results

In [40]:
restored_run.stop()


Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 1 operations to synchronize with Neptune. Do not kill this process.
All 1 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/thingumajig/ner-training/e/NERTRAIN-4
